In [1]:
import tensorflow as tf

In [3]:
sales_num = [29,24,-109,78,9,120,-1,34]
tf_dataset = tf.data.Dataset.from_tensor_slices(sales_num)
tf_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [5]:
for item in tf_dataset.as_numpy_iterator():
    print(item)

29
24
-109
78
9
120
-1
34


In [6]:
# convert numbers from USD to INR
tf_dataset  = tf_dataset.map(lambda x: x*87)

for item in tf_dataset.as_numpy_iterator():
    print(item)

2523
2088
-9483
6786
783
10440
-87
2958


In [ ]:
#  shuffle
tf_dataset = tf_dataset.shuffle(2)
for item in tf_dataset.as_numpy_iterator():
    print(item)

-9483
2958
783
10440
2088
2523
-87
6786


In [21]:
#  batching
for item in tf_dataset.batch(3).as_numpy_iterator():
    print(item)

[2523 2088  783]
[ 2958   -87 10440]
[ 6786 -9483]


In [26]:
# perform all prev operations in one line
tf_dataset = tf.data.Dataset.from_tensor_slices(sales_num)

tf_dataset = tf_dataset.map(lambda x: x*87).shuffle(2).batch(3)
for item in tf_dataset.as_numpy_iterator():
    print(item)

[2088 2523 6786]
[  783 -9483 10440]
[ -87 2958]


In [27]:
img_data = tf.data.Dataset.list_files('images/*/*',shuffle=False)

In [29]:
img_count = len(list(img_data))
img_count

130

In [30]:
type(img_data)

tensorflow.python.data.ops.from_tensor_slices_op._TensorSliceDataset

In [31]:
for img in img_data.take(5):
    print(img)

tf.Tensor(b'images\\cat\\20 Reasons Why Cats Make the Best Pets....jpg', shape=(), dtype=string)
tf.Tensor(b'images\\cat\\7 Foods Your Cat Can_t Eat.jpg', shape=(), dtype=string)
tf.Tensor(b'images\\cat\\A cat appears to have caught the....jpg', shape=(), dtype=string)
tf.Tensor(b'images\\cat\\Adopt-A-Cat Month\xc2\xae - American Humane....jpg', shape=(), dtype=string)
tf.Tensor(b'images\\cat\\All About Your Cat_s Tongue.jpg', shape=(), dtype=string)


In [36]:
img_data = img_data.shuffle(200)
for img in img_data.take(5):
    print(img)

tf.Tensor(b'images\\dog\\45 Best Large Dog Breeds - Top Big Dogs_yyth....jpg', shape=(), dtype=string)
tf.Tensor(b'images\\cat\\Cats really do need their humans_ even....jpg', shape=(), dtype=string)
tf.Tensor(b'images\\cat\\How to buy the best cat food_ according....jpg', shape=(), dtype=string)
tf.Tensor(b'images\\dog\\scientists explain puppy dog eyes....jpg', shape=(), dtype=string)
tf.Tensor(b'images\\cat\\All About Your Cat_s Tongue.jpg', shape=(), dtype=string)


In [37]:
class_names = ['dog','cat']

In [39]:
train_size = int(img_count * 0.8)
train_data = img_data.take(train_size)
test_data = img_data.skip(train_size)
len(train_data), len(test_data)

(104, 26)

In [41]:
def get_label(path):
    import os
    parts = tf.strings.split(path, os.path.sep)
    return parts[-2]

In [42]:
get_label('images\\dog\\45 Best Large Dog Breeds - Top Big Dogs_yyth....jpg')

<tf.Tensor: shape=(), dtype=string, numpy=b'dog'>

In [45]:
def process_img(path):
    label = get_label(path)
    img = tf.io.read_file(path)
    img = tf.io.decode_jpeg(img)
    img = tf.image.resize(img, [128,128])
    return img,label

In [48]:
img, label = process_img('images\\dog\\45 Best Large Dog Breeds - Top Big Dogs_yyth....jpg')
img.numpy()[:2]

array([[[224.01953  , 223.01953  ,  55.01953  ],
        [236.98047  , 235.98047  ,  58.01953  ],
        [219.13672  , 216.13672  ,  43.097656 ],
        [217.09766  , 214.09766  ,  41.097656 ],
        [229.03906  , 223.03906  ,  67.       ],
        [224.07812  , 212.07812  ,  92.078125 ],
        [210.09766  , 195.09766  ,  80.05859  ],
        [232.11719  , 222.13672  ,  72.078125 ],
        [203.0586   , 195.0586   ,  34.01953  ],
        [200.07812  , 189.07812  ,  47.078125 ],
        [202.03906  , 189.03906  ,  48.058594 ],
        [208.96094  , 196.9414   ,  51.039062 ],
        [217.96094  , 206.9414   ,  55.01953  ],
        [206.       , 193.       ,  52.01953  ],
        [201.92188  , 183.92188  ,  59.921875 ],
        [199.01953  , 179.       ,  58.039062 ],
        [214.98047  , 202.98047  ,  68.94141  ],
        [228.       , 215.01953  ,  83.96094  ],
        [233.11719  , 218.11719  ,  99.15625  ],
        [233.       , 225.       ,  88.       ],
        [240.98047  

In [50]:
train_data = train_data.map(process_img)
test_data = test_data.map(process_img)

In [51]:
for img, label in train_data.take(1):
    print("image:", img)
    print("label:", label)

image: tf.Tensor(
[[[ 71.5       70.5       75.5     ]
  [ 62.5       61.916016  65.66797 ]
  [ 53.416016  55.416016  52.416016]
  ...
  [ 31.5       39.5       15.5     ]
  [ 35.83203   44.83203   25.832031]
  [ 43.916016  54.916016  40.916016]]

 [[ 72.00391   71.00391   76.00391 ]
  [ 59.503906  59.503906  59.503906]
  [ 48.003906  50.003906  45.003906]
  ...
  [ 33.748047  38.748047  15.748047]
  [ 37.992188  42.503906  31.      ]
  [ 45.003906  50.003906  43.003906]]

 [[ 68.5       66.5       71.5     ]
  [ 55.08008   55.08008   53.08008 ]
  [ 48.5       50.5       45.5     ]
  ...
  [ 36.740234  39.740234  18.740234]
  [ 40.240234  44.240234  30.240234]
  [ 43.58008   48.58008   41.58008 ]]

 ...

 [[155.51953  181.        98.59961 ]
  [144.25977  172.25977   87.67969 ]
  [149.82031  175.82031   86.66016 ]
  ...
  [122.5      150.5       66.5     ]
  [128.       156.        81.09961 ]
  [135.66016  162.5       95.82031 ]]

 [[173.74805  186.49219  119.74805 ]
  [163.76367  177.7

In [53]:
def scale(img, label):
    return img/255, label

In [56]:
train_data = train_data.map(scale)

In [ ]:
for img, label in train_data.take(1):
    print("image:", img)
    print("label:", label)

image: tf.Tensor(
[[[0.40784314 0.4117647  0.39215687]
  [0.4117647  0.41568628 0.39607844]
  [0.42352942 0.41960785 0.40392157]
  ...
  [0.9372549  0.9411765  0.92156863]
  [0.9372549  0.9411765  0.92156863]
  [0.9372549  0.9411765  0.92156863]]

 [[0.40801546 0.41193703 0.3923292 ]
  [0.41453356 0.41845512 0.39884728]
  [0.41568628 0.41960785 0.4       ]
  ...
  [0.9372549  0.9411765  0.92156863]
  [0.9372549  0.9411765  0.92156863]
  [0.9372549  0.9411765  0.92156863]]

 [[0.40784314 0.40784314 0.4       ]
  [0.4117647  0.41568628 0.39607844]
  [0.41960785 0.41568628 0.4       ]
  ...
  [0.9372549  0.9411765  0.92156863]
  [0.9372549  0.9411765  0.92156863]
  [0.9372549  0.9411765  0.92156863]]

 ...

 [[0.29689032 0.24590993 0.21061581]
  [0.34139094 0.28648898 0.24335171]
  [0.2767578  0.22223882 0.1869447 ]
  ...
  [0.49832645 0.4904833  0.49440488]
  [0.5024395  0.49851793 0.48990884]
  [0.49842218 0.4945006  0.48665747]]

 [[0.35559514 0.30069315 0.26539904]
  [0.3426164  0.287